# Paillier Crypto

## Generate Public and Private Key

In [1]:
import os

import torch

from openfed.functional import PrivateKey, key_gen

if not os.path.isfile('/tmp/public.key') or not os.path.isfile(
        '/tmp/private.key'):
    public_key, private_key = key_gen()
    public_key.save('/tmp/public.key')
    private_key.save('/tmp/private.key')
    print("Save new key to /tmp/public.key and /tmp/private.key")
else:
    private_key = PrivateKey.load('/tmp/private.key')
    print("Load private key from /tmp/private.key")
print(private_key)

Load private key from /tmp/private.key
<OpenFed> PrivateKey
+-------+------+-----------+----+-------+------------+------------+
| n_lwe | bits | bits_safe | l  | bound |     p      |     q      |
+-------+------+-----------+----+-------+------------+------------+
|  3000 |  32  |     24    | 64 |   8   | 4294967297 | 4294967296 |
+-------+------+-----------+----+-------+------------+------------+



## Network

In [2]:
import torch.nn as nn

network = nn.Linear(784, 10)
loss_fn = nn.CrossEntropyLoss()

## Optimizer

In [3]:
import torch
from openfed.federated import leader
from openfed.optim import FederatedOptimizer

optim = torch.optim.SGD(network.parameters(), lr=1.0)
fed_optim = FederatedOptimizer(optim, role=leader)
print(fed_optim)

<OpenFed> FederatedOptimizer
SGD (
Parameter Group 0
    dampening: 0
    lr: 1.0
    momentum: 0
    nesterov: False
    weight_decay: 0
)



## Topology

In [4]:
from openfed.topo import Node, Topology, analysis
import openfed

server_node = Node('server', openfed.default_tcp_address)
client = Node('client', openfed.empty_address)

topology = Topology()
topology.add_edge(client, server_node)

fed_props = analysis(topology, server_node)[0]

print(fed_props)

<OpenFed> FederatedProperties
+----------------+-----------+
|      role      | nick_name |
+----------------+-----------+
| openfed_leader |   server  |
+----------------+-----------+
<OpenFed> Address
+---------+---------------------+------------+------+
| backend |     init_method     | world_size | rank |
+---------+---------------------+------------+------+
|   gloo  | tcp://localhost:... |     2      |  0   |
+---------+---------------------+------------+------+




## Maintainer

In [5]:
from openfed.core import Maintainer

mt = Maintainer(fed_props, network.state_dict(keep_vars=True))

with mt:
    openfed.F.device_alignment()
    openfed.F.count_step(2)

print(mt)

<OpenFed> Maintainer
+----------------+-----------+-------+
|      role      | nick_name | pipes |
+----------------+-----------+-------+
| openfed_leader |   server  |   1   |
+----------------+-----------+-------+



# Step

In [6]:
openfed.api(mt,
            fed_optim,
            5,
            agg_func=openfed.F.paillier_aggregation,
            private_key=private_key)

100%|██████████| 5/5 [00:08<00:00,  1.78s/it]
